In [1]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/hubble/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/hubble/')
import os
os.environ['PATH'] = '/Users/ravi.tej/anaconda3/envs/bertopicenv/bin:/Users/ravi.tej/anaconda3/condabin:/usr/bin:/bin:/usr/sbin:/sbin'

In [5]:
from openai import OpenAI
import json

client = OpenAI(api_key=os.environ.get('OPENAI_KEY'))

In [124]:
sys = '''
You are an expert financial analyst working for a leading indian fintech platform. You extract the following attributes from a given user query. Today is “2024-03-15”. The platform has a database with the following data for indian NSE stocks.
stock_history: symbol, date, open, close
Using the companies names, the following query_types are available:
1. get_stocks_price_between_dates(symbol_list, time_period=‘3 months’): to get the day wise close prices of a list of stock names. 
2. get_avg_stock_prices_between_dates(symbol_list, time_period=‘3 months’): to get the average close prices of a list of stock names between two dates. 
3. beta_calculation_query(symbols, time_period=‘3 months’): beta of a list of symbols
is_financial_query (True/False), sql_db_has_data(True/False), companies_mentioned (list of strings), time_period_type (sql compatible), time_period_duration(1-3), query_groups(one of get_stocks_price_between_dates, get_avg_stock_prices_between_dates, beta_calculation_query)

companies_mentioned, time_period_type, time_period_duration, query_groups are all related to the sql search. so if sql_db_has_data is True, then all of them must be present
respond only with the json with the above keys. don't answer the user query directly, but only extract the attributes
'''

In [45]:
'''
examples query and response:

How to cook biryani: False,,,,,
Latest budget updates: True, web,,,,
Current bitcoin price: True, web,,,,
Adani stock price: True, sql,[‘Adani’],month,3,get_stock_prices_between_dates
Compare TCS, reliance and Paytm performance for the last year: True, sql,[‘TCS’, ‘reliance’, ‘Paytm’],year, 1,get_stock_prices_between_dates
What is the volatility of zomoto and ITC last week: True, sql,[‘zomato’, ‘ITC’],week, 1,beta_calculation_query
'''

'\nexamples query and response:\n\nHow to cook biryani: False,,,,,\nLatest budget updates: True, web,,,,\nCurrent bitcoin price: True, web,,,,\nAdani stock price: True, sql,[‘Adani’],month,3,get_stock_prices_between_dates\nCompare TCS, reliance and Paytm performance for the last year: True, sql,[‘TCS’, ‘reliance’, ‘Paytm’],year, 1,get_stock_prices_between_dates\nWhat is the volatility of zomoto and ITC last week: True, sql,[‘zomato’, ‘ITC’],week, 1,beta_calculation_query\n'

In [46]:
# def get_query_details(financial_or_business_query, sql_db_has_data,companies_mentioned,time_period, function_name):
    

In [134]:
messages = [{"role": "system", "content": sys}, 
            {"role": "user", "content": "user_query: what is the ideal XIRR for a mutual fund portfolio"}]
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_query_details",
            "description": "Get details of user query",
            "parameters": {
                "type": "object",
                "properties": {
                    "financial_or_business_query": {"type": "boolean","description": "If the query is relevant for personal finance, markets of business related queries"},
                    "sql_db_has_data": {"type": "boolean", "description": "If running a sql query over stock_history table will answer user query"},
                    "companies_mentioned": {"type": "array", "items": {"type": "string"}, "minItems": 1, "maxItems": 5, "description": "Array of company names mentioned in the user query"},
                    "time_period_type": {"type": "string", "enum": ["year", "month", "week", "day"]},
                    "time_period_duration": {"type": "integer", "minimum": 1, "maximum": 11},
                    "query_type": {"type": "string", "enum": ["get_stock_prices_between_dates", "get_avg_stock_prices_between_dates", "beta_calculation_query"]}
                },
                "required": ["financial_or_business_query", "sql_db_has_data"],
            },
        },
    }
]
# 
# gpt-4-0125-preview
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=messages,
    tools=tools,
    tool_choice={'type': 'function', 'function': {'name': 'get_query_details'}},  # auto is default, but we'll be explicit
)

In [25]:
# r = []

In [60]:
r.append(response)

In [61]:
response.choices[0].message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_9k77KbApMzAbjLddQPvYEDWT', function=Function(arguments='{"financial_or_business_query":true,"sql_db_has_data":false}', name='get_query_details'), type='function')])

In [93]:
json.loads(response.choices[0].message.tool_calls[0].function.arguments)

{'financial_or_business_query': True, 'sql_db_has_data': False}

In [65]:
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [67]:
def get_all_symbol_names():
    with PostgresDatabaseOperation() as cursor:
        sql = 'SELECT DISTINCT symbol, name FROM stock_ticker_info'
        cursor.execute(sql)
        results = cursor.fetchall()
    
    return results

In [68]:
def get_all_company_wise_volume():
    with PostgresDatabaseOperation() as cursor:
        sql = "SELECT symbol, AVG(volume * close) FROM stock_history WHERE symbol LIKE '%.NS' AND date > '2024-01-01' GROUP BY symbol"
        cursor.execute(sql)
        results = cursor.fetchall()
    return results

In [69]:
avg_volume_details = get_all_company_wise_volume()

In [70]:
import numpy as np

In [71]:
volume_dict = {x[0]: {'abs': float(x[1]), 'log': np.log2(float(x[1]))} for x in avg_volume_details}

/var/folders/d4/cgyr_gnj7nn2wy_hq40gkq8c0000gq/T/ipykernel_35623/1270495777.py:1: RuntimeWarning: divide by zero encountered in log2
  volume_dict = {x[0]: {'abs': float(x[1]), 'log': np.log2(float(x[1]))} for x in avg_volume_details}


In [72]:
import pandas as pd

In [73]:
volume_df = pd.DataFrame(avg_volume_details, columns = ['symbol', 'avg_volume']).set_index('symbol').reset_index()

In [74]:
volume_df['log_volume'] = volume_df['avg_volume'].apply(lambda x: np.log2(float(x))/np.log2(1.5) if float(x) > 1 else 1)

In [75]:
volume_df

,symbol,avg_volume,log_volume
0,20MICRONS.NS,30719768.17615295508091,42.520100
1,21STCENMGM.NS,301008.914794998170304,31.112160
2,3IINFOLTD.NS,161305952.576985397372464,46.610209
3,3MINDIA.NS,160494037.199101563,46.597764
4,3PLAND.NS,2059614.987348632803598,35.855193
...,...,...,...
1799,ZODIAC.NS,5687203.40596631106407,38.360216
1800,ZOMATO.NS,9866556384.38835288761918,56.755603
1801,ZOTA.NS,18296907.87303894009319,41.242125
1802,ZUARI.NS,153157835.41733276448707,46.482371


In [76]:
symbol_names = get_all_symbol_names()

In [77]:
from fuzzywuzzy import process, fuzz

In [78]:
name_to_symbol = {}

In [79]:
for sym, name in symbol_names:
    if sym.split('.')[1] == 'NS':
        if name in symbol_names:
            name_to_symbol[name].append(sym)
        else:
            name_to_symbol[name] = [sym]

In [80]:
len(name_to_symbol)

1931

In [85]:
distinct_names = list(name_to_symbol.keys())

In [86]:
distinct_symbols = [x[0] for x in symbol_names if '.NS' in x[0]]

In [104]:
import requests

In [153]:
def orchestrate(query):
    query_details = parse_details_from_gpt(query)
    # print(query_details)
    if not query_details['financial_or_business_query']:
        print('not a financial query')
        return
    if not query_details['sql_db_has_data']:
        r = requests.post('http://127.0.0.1:8000/search_query_web', json={'query_text': query})
        return r
    for mandatory_key in ['companies_mentioned', 'time_period_type', 'time_period_duration', 'query_type']:
        if mandatory_key not in query_details:
            print('ill formed arguments. calling web instead')
            r = requests.post('http://127.0.0.1:8000/search_query_web', json={'query_text': query})
            return r
    symbols = []
    for name in query_details['companies_mentioned']:
        symbols.append(get_symbol_for_name(name))
    time_period = f"{query_details['time_period_duration']} {query_details['time_period_type']}"
    function_mapping = {'get_stocks_price_between_dates': get_stocks_price_between_dates, 'get_avg_stock_prices_between_dates': get_avg_stock_prices_between_dates, 'beta_calculation_query': beta_calculation_query}
    function_to_call = function_mapping[query_details['query_type']]
    results = function_to_call(symbols, time_period)
    return results

In [154]:
r = orchestrate('which among airtel adani reliance and apollo stocks had the best performance')

In [155]:
r

[('ADANIENT.NS', Decimal('3086.9669541181144068')),
 ('APLAPOLLO.NS', Decimal('1499.1050797801906780')),
 ('BHARTIARTL.NS', Decimal('1106.6203344312764831')),
 ('RELIANCE.NS', Decimal('2799.7644084348516949'))]

In [145]:
r.text

'{"additionalInfo":{"query_understanding":{"completion_tokens":17,"evaluation":"True,High,False,is gold a good investment right now 24/03/23","model":"gpt-4-0125-preview","prompt_tokens":208,"system_prompt":"You are an expert in taking user queries on an indian financial and business website and extracting important attributes separated by comma. The final attribute of web search query is extremely important, and should directly result in relevant results when the query has any financial or business connection. If the entity or query is too vague to get only financial or business results, then append it with \\u2018financial and business news\\u2019. Today is 18/03/24. When using in web query, then use %d/%m/%y\\n- is_fin_business_query(True/False), recency_importance (High for very volatile, medium for slowly changing stuff like regulations or industry related, and low for very less changing like educational and guides), vague(True/False), web_search_query\\nexample query: best credit

In [152]:
def parse_details_from_gpt(query):
    messages = [{"role": "system", "content": sys}, 
            {"role": "user", "content": f"user_query: {query}"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_query_details",
                "description": "Get details of user query",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "financial_or_business_query": {"type": "boolean","description": "If the query is relevant for personal finance, markets of business related queries"},
                        "sql_db_has_data": {"type": "boolean", "description": "If running a sql query over stock_history table will answer user query"},
                        "companies_mentioned": {"type": "array", "items": {"type": "string"}, "minItems": 0, "maxItems": 5, "description": "Array of company names mentioned in the user query"},
                        "time_period_type": {"type": "string", "enum": ["year", "month", "week", "day"]},
                        "time_period_duration": {"type": "integer", "minimum": 1, "maximum": 3},
                        "query_type": {"type": "string", "enum": ["get_stocks_price_between_dates", "get_avg_stock_prices_between_dates", "beta_calculation_query"]}
                    },
                    "required": ["financial_or_business_query", "sql_db_has_data", "companies_mentioned", "time_period_type", "time_period_duration", "query_type"],
                },
            },
        }
    ]
    # 
    # gpt-4-0125-preview
    response = client.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=messages,
        tools=tools,
        tool_choice={'type': 'function', 'function': {'name': 'get_query_details'}},  # auto is default, but we'll be explicit
    )
    return json.loads(response.choices[0].message.tool_calls[0].function.arguments)

In [102]:
def get_symbol_for_name(name):
    k = process.extract(name, distinct_names)
    symbols_matched = []
    for name_match, score in k:
        symbols_matched.extend([(cur_symbol,score) for cur_symbol in name_to_symbol[name_match]])
    p = process.extract(name, distinct_symbols)
    # sym = name_to_symbol[matching_name][0]
    matches = pd.DataFrame(symbols_matched + p, columns = ['symbol', 'matching_score'])
    matches = pd.merge(matches, volume_df, how = 'left', on = 'symbol')
    matches['score'] = matches['matching_score'] * matches['log_volume']
    return matches[matches['matching_score'] > 75].nlargest(1, 'score')['symbol'].values[0]

In [103]:
get_symbol_for_name('paytm')

'PAYTM.NS'

In [133]:
def get_stocks_price_between_dates(symbols, time_period='3 month'):
    symbol_string = ''
    for sym in symbols:
        symbol_string = symbol_string + f"'{sym}',"
    symbol_string = symbol_string.strip(',')
    with PostgresDatabaseOperation() as cursor:
        sql = f"SELECT date, symbol, close FROM stock_history_latest WHERE symbol in ({symbol_string}) AND date >= CURRENT_DATE - INTERVAL '{time_period}'"
        cursor.execute(sql)
        results = cursor.fetchall()
    return results

def get_avg_stock_prices_between_dates(symbols, time_period='3 month'):
    symbol_string = ''
    for sym in symbols:
        symbol_string = symbol_string + f"'{sym}',"
    symbol_string = symbol_string.strip(',')
    with PostgresDatabaseOperation() as cursor:
        sql = f"SELECT symbol, avg(close) FROM stock_history_latest WHERE symbol in ({symbol_string}) AND date >= CURRENT_DATE - INTERVAL '{time_period}' GROUP BY symbol"
        cursor.execute(sql)
        results = cursor.fetchall()
    return results

def beta_calculation_query(symbols, time_period='1 year'):
    market_symbol = '^NSEI'  # NSE Index symbol
    symbol_string = ','.join(f"'{sym}'" for sym in symbols)

    stock_returns_subquery = f"""
    SELECT symbol, 
           date, 
           (close - LAG(close, 1) OVER (PARTITION BY symbol ORDER BY date)) / LAG(close, 1) OVER (PARTITION BY symbol ORDER BY date) AS daily_return
    FROM stock_history
    WHERE symbol IN ({symbol_string}) AND date >= CURRENT_DATE - INTERVAL '{time_period}'
    """

    market_returns_subquery = f"""
    SELECT date, 
           (close - LAG(close, 1) OVER (ORDER BY date)) / LAG(close, 1) OVER (ORDER BY date) AS daily_return
    FROM stock_history
    WHERE symbol = '{market_symbol}' AND date >= CURRENT_DATE - INTERVAL '{time_period}'
    """

    # Assuming you want to calculate beta for each stock symbol against the market index
    beta_query = f"""
    SELECT s.symbol, 
           (COVAR_SAMP(s.daily_return, m.daily_return) / VAR_SAMP(m.daily_return)) AS beta
    FROM ({stock_returns_subquery}) s
    INNER JOIN ({market_returns_subquery}) m ON s.date = m.date
    GROUP BY s.symbol
    """
    
    with PostgresDatabaseOperation() as cursor:
        cursor.execute(beta_query)
        results = cursor.fetchall()
    return results

### User investments

In [156]:
import pandas as pd

In [157]:
df = pd.read_csv('user_investments.csv')

In [161]:
df = df[['user_id', 'investment_option_id', 'quantity', 'average_price', 'isin', 'name', 'nseticker', 'bseticker']]

In [173]:
df['nseticker'] = df['nseticker'].apply(lambda x: x + '.NS' if '.NS' not in x else x)
df['bseticker'] = df['bseticker'].apply(lambda x: x + '.BO' if pd.notnull(x) and '.BO' not in x else x)

/var/folders/d4/cgyr_gnj7nn2wy_hq40gkq8c0000gq/T/ipykernel_35623/850385422.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['nseticker'] = df['nseticker'].apply(lambda x: x + '.NS' if '.NS' not in x else x)
/var/folders/d4/cgyr_gnj7nn2wy_hq40gkq8c0000gq/T/ipykernel_35623/850385422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bseticker'] = df['bseticker'].apply(lambda x: x + '.BO' if pd.notnull(x) and '.BO' not in x else x)


In [174]:
df

,user_id,investment_option_id,quantity,average_price,isin,name,nseticker,bseticker
0,1,1,12,598.375000,INE699H01024,Adani Wilmar Ltd,AWL.NS,AWL.BO
1,1,2,15,613.720000,INE118H01025,BSE Ltd,BSE.NS,NaN
2,1,3,10,1187.260000,INE736A01011,Central Depository Services (India) Ltd,CDSL.NS,NaN
3,1,4,4,1576.300000,INE259A01022,Colgate-Palmolive (India) Ltd,COLPAL.NS,COLPAL.BO
4,1,5,50,239.270000,INE509F01011,Cupid Ltd,CUPID.NS,CUPID.BO
5,1,6,204,46.186617,INF204KB17I5,Nippon India ETF Gold BeES,GOLDBEES.NS,GOLDBEES.BO
6,1,7,5,429.500000,INE08ZM01014,Greenpanel Industries Ltd,GREENPANEL.NS,GREENPANEL.BO
7,1,8,60,157.880000,INE022Q01020,Indian Energy Exchange Ltd,IEX.NS,IEX.BO
8,1,9,15,696.270000,INE335Y01020,Indian Railway Catering and Tourism Corporatio...,IRCTC.NS,IRCTC.BO
9,1,10,40,348.560000,INE154A01025,Itc Ltd,ITC.NS,ITC.BO


In [175]:
from io import StringIO
output = StringIO()

In [176]:
df.to_csv(output, sep='\t', header=False, index=False)

In [177]:
output.seek(0)

0

In [178]:
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [179]:
import time

In [180]:
s = time.time()
with PostgresDatabaseOperation() as cursor:
    cursor.copy_from(output, 'user_investments_temp', null='')
e = time.time()

### Portfolio Queries

In [195]:
def current_portfolio_value():
    sql = '''SELECT name, SUM(quantity * close) FROM user_investments_temp uit LEFT JOIN stock_history_latest sh
            ON uit.nseticker = sh.symbol
            WHERE date = '2024-03-10 18:30:00'
            GROUP BY name
    '''
    with PostgresDatabaseOperation() as cursor:
        cursor.execute(sql)
        r = cursor.fetchall()
    df = pd.DataFrame([(x[0], float(x[1])) for x in r], columns = ['stock', 'current_value'])
    total_value = df['current_value'].sum()
    df['percent'] = 100 * df['current_value'] / total_value
    df = df.sort_values('percent',ascending = False)
    return df, total_value

In [220]:
def compare_portfolio_performance_with_index():
    sql = '''
    WITH initial_values AS (
    SELECT
        'portfolio' AS type,
        SUM(ui.quantity * sh.close) AS initial_value
    FROM
        user_investments_temp ui
    JOIN
        stock_history_latest sh ON ui.nseticker = sh.symbol
    WHERE
        sh.date = (SELECT MIN(date) FROM stock_history_latest WHERE date >= CURRENT_DATE - INTERVAL '6 months')
    UNION ALL
    SELECT
        'index' AS type,
        sh.close AS initial_value
    FROM
        stock_history_latest sh
    WHERE
        sh.symbol = '^NSEI' AND
        sh.date = (SELECT MIN(date) FROM stock_history_latest WHERE date >= CURRENT_DATE - INTERVAL '6 months')
),
daily_values AS (
    SELECT
        sh.date,
        'portfolio' AS type,
        SUM(ui.quantity * sh.close) AS current_value
    FROM
        user_investments_temp ui
    JOIN
        stock_history_latest sh ON ui.nseticker = sh.symbol
    WHERE
        sh.date >= CURRENT_DATE - INTERVAL '6 months'
    GROUP BY
        sh.date
    UNION ALL
    SELECT
        sh.date,
        'index' AS type,
        sh.close AS current_value
    FROM
        stock_history_latest sh
    WHERE
        sh.symbol = '^NSEI' AND
        sh.date >= CURRENT_DATE - INTERVAL '6 months'
),
growth AS (
    SELECT
        dv.date,
        dv.type,
        dv.current_value,
        iv.initial_value,
        (dv.current_value - iv.initial_value) / iv.initial_value * 100 AS growth_percentage
    FROM
        daily_values dv
    JOIN
        initial_values iv ON dv.type = iv.type
)
SELECT
    date,
    MAX(CASE WHEN type = 'portfolio' THEN growth_percentage ELSE NULL END) AS portfolio_growth_percentage,
    MAX(CASE WHEN type = 'index' THEN growth_percentage ELSE NULL END) AS index_growth_percentage
FROM
    growth
GROUP BY
    date
ORDER BY
    date;
    '''
    with PostgresDatabaseOperation() as cursor:
        cursor.execute(sql)
        r = cursor.fetchall()
    df = pd.DataFrame([(x[0], float(x[1]), float(x[2])) if x[1] and x[2] else (x[0], None, None) for x in r], columns = ['stock', 'portfolio', 'nse_index'])
    # total_value = df['current_value'].sum()
    # df['percent'] = 100 * df['current_value'] / total_value
    # df = df.sort_values('percent',ascending = False)
    return df

In [221]:
k3 = compare_portfolio_performance_with_index()

In [222]:
k3

,stock,portfolio,nse_index
0,2023-09-19 18:30:00+00:00,NaN,NaN
1,2023-09-20 18:30:00+00:00,-0.599073,-0.799194
2,2023-09-21 18:30:00+00:00,-0.715371,-1.141379
3,2023-09-24 18:30:00+00:00,-0.273710,-1.139868
4,2023-09-25 18:30:00+00:00,0.797870,-1.189369
...,...,...,...
114,2024-03-06 18:30:00+00:00,80.421672,13.024965
115,2024-03-10 18:30:00+00:00,76.142144,12.216477
116,2024-03-11 18:30:00+00:00,77.008872,12.231797
117,2024-03-12 18:30:00+00:00,69.446743,10.533424


In [237]:
def calculate_portfolio_growth_for_period(time_period = '4 week'):
    sql = f'''
    WITH initial_value AS (
    SELECT
        SUM(ui.quantity * sh.close) AS initial_value
    FROM
        user_investments_temp ui
    JOIN
        stock_history_latest sh ON ui.nseticker = sh.symbol
    WHERE
        sh.date = (SELECT MIN(date) FROM stock_history_latest WHERE date >= CURRENT_DATE - INTERVAL '{time_period}')
),
daily_value AS (
    SELECT
        sh.date,
        SUM(ui.quantity * sh.close) AS current_value
    FROM
        user_investments_temp ui
    JOIN
        stock_history_latest sh ON ui.nseticker = sh.symbol
    WHERE
        sh.date >= CURRENT_DATE - INTERVAL '{time_period}'
    GROUP BY
        sh.date
),
growth AS (
    SELECT
        dv.date,
        iv.initial_value,
        dv.current_value,
        (dv.current_value - iv.initial_value) / iv.initial_value * 100 AS growth_percentage
    FROM
        daily_value dv
    CROSS JOIN
        initial_value iv
)
SELECT
    date,
    current_value,
    growth_percentage
FROM
    growth
ORDER BY
    date;
    '''
    with PostgresDatabaseOperation() as cursor:
        cursor.execute(sql)
        r = cursor.fetchall()
    df = pd.DataFrame([(x[0], float(x[1]), float(x[2])) if x[2] else (x[0], x[1], x[2]) for x in r], columns = ['date', 'value', 'growth_percent'])
    # total_value = df['current_value'].sum()
    # df['percent'] = 100 * df['current_value'] / total_value
    # df = df.sort_values('percent',ascending = False)
    return df

In [238]:
k4 = calculate_portfolio_growth_for_period()

In [239]:
k4

,date,value,growth_percent
0,2024-02-20 18:30:00+00:00,371249.57403564453088,0E-24
1,2024-02-21 18:30:00+00:00,372818.047028,0.422485
2,2024-02-22 18:30:00+00:00,379581.00441,2.244159
3,2024-02-25 18:30:00+00:00,384930.777985,3.685177
4,2024-02-26 18:30:00+00:00,393620.29599,6.025791
5,2024-02-27 18:30:00+00:00,396161.648804,6.710331
6,2024-02-28 18:30:00+00:00,398284.291412,7.282087
7,2024-02-29 18:30:00+00:00,402119.906464,8.315251
8,2024-03-03 18:30:00+00:00,393994.025787,6.126459
9,2024-03-04 18:30:00+00:00,390030.573441,5.058861


In [249]:
def top_gainers():
    sql = '''
    WITH initial_prices AS (
    SELECT
        sh.symbol AS nseticker,
        sh.close AS initial_price
    FROM
        stock_history_latest sh
    WHERE
        sh.date = (SELECT MIN(date) FROM stock_history_latest WHERE EXTRACT(YEAR FROM date) = EXTRACT(YEAR FROM CURRENT_DATE))
),
current_prices AS (
    SELECT
        sh.symbol AS nseticker,
        sh.close AS current_price
    FROM
        stock_history_latest sh
    WHERE
        sh.date = (SELECT MAX(date) FROM stock_history_latest WHERE EXTRACT(YEAR FROM date) = EXTRACT(YEAR FROM CURRENT_DATE))
),
investment_values AS (
    SELECT
        ui.nseticker,
        ui.quantity * ip.initial_price AS initial_value,
        ui.quantity * cp.current_price AS current_value
    FROM
        user_investments_temp ui
    JOIN
        initial_prices ip ON ui.nseticker = ip.nseticker
    JOIN
        current_prices cp ON ui.nseticker = cp.nseticker
),
value_change AS (
    SELECT
        iv.nseticker,
        iv.current_value - iv.initial_value AS absolute_change,
        ((cp.current_price - ip.initial_price) / ip.initial_price * 100) AS percentage_change
    FROM
        investment_values iv
    JOIN
        initial_prices ip ON iv.nseticker = ip.nseticker
    JOIN
        current_prices cp ON iv.nseticker = cp.nseticker
)
SELECT
    nseticker,
    absolute_change,
    percentage_change
FROM
    value_change
ORDER BY
    absolute_change DESC;
    '''
    with PostgresDatabaseOperation() as cursor:
        cursor.execute(sql)
        r = cursor.fetchall()
    df = pd.DataFrame([(x[0], float(x[1]),float(x[2])) for x in r], columns = ['date', 'absolute', 'price_percent'])
    # total_value = df['current_value'].sum()
    # df['percent'] = 100 * df['current_value'] / total_value
    # df = df.sort_values('percent',ascending = False)
    return df

In [250]:
k5 = top_gainers()

In [253]:
k5

,date,absolute,price_percent
0,CUPID.NS,54137.506104,95.283156
1,ONGC.NS,2830.705566,26.676346
2,TATAPOWER.NS,1920.600220,16.174020
3,ZOMATO.NS,1062.600403,18.764576
4,COLPAL.NS,629.199219,6.285455
5,PIDILITIND.NS,529.749756,3.870035
6,JUNIORBEES.NS,359.939880,9.073901
7,MCX.NS,323.000488,1.016348
8,NIFTYBEES.NS,136.319824,2.374480
9,IRCTC.NS,101.999817,0.767320


In [252]:
k5['absolute'].sum()

51477.20962524414

In [212]:
k3.portfolio.cumsum()

0              NaN
1        -0.599073
2        -1.314444
3        -1.588153
4        -0.790283
          ...     
114    4337.851133
115    4413.993277
116    4491.002149
117    4560.448892
118    4607.252529
Name: portfolio, Length: 119, dtype: float64

In [213]:
k3.nse_index.cumsum()

0             NaN
1       -0.799194
2       -1.940573
3       -3.080440
4       -4.269810
          ...    
114    508.384087
115    520.600564
116    532.832361
117    543.365785
118    554.647654
Name: nse_index, Length: 119, dtype: float64

In [191]:
k1, k2 = current_value()

In [194]:
k1.to_dict('list')

{'stock': ['Cupid Ltd',
  'Multi Commodity Exchange of India Ltd',
  'BSE Ltd',
  'Tata Motors Ltd',
  'Central Depository Services (India) Ltd',
  'Itc Ltd',
  'Titan Company Ltd',
  'Tata Power Company Ltd',
  'Pidilite Industries Ltd',
  'Oil and Natural Gas Corporation Ltd',
  'Indian Railway Catering and Tourism Corporation Ltd',
  'Nippon India ETF Gold BeES',
  'Colgate-Palmolive (India) Ltd',
  'MTAR Technologies Ltd',
  'Indian Energy Exchange Ltd',
  'Nippon India ETF Nifty 1D Rate Liquid BeES',
  'Zomato Ltd',
  'Nippon India ETF Nifty 50 BeES',
  'Nippon India ETF Nifty Next 50 Junior BeES',
  'Greenpanel Industries Ltd'],
 'current_value': [117087.5,
  35327.5,
  33061.50146484375,
  20560.0,
  18105.999755859375,
  16375.999755859377,
  15000.599609375,
  14873.399780273438,
  14531.99951171875,
  14248.0,
  14078.999633789062,
  11426.039657592773,
  10427.400390625,
  8966.749877929688,
  8616.000366210938,
  8000.0,
  6813.4002685546875,
  5938.7999267578125,
  4497.42